In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import lsst.sims.maf.utils as mafUtils
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.plots as plots



In [ ]:
%pylab inline

In [4]:
runName = 'minion_1016'
dbfile = '/global/homes/h/husni/lsstpsf/simruns/minion_1012_sqlite.db'
opsdb = db.OpsimDatabase(dbfile)

In [24]:
class KuiperMetric(metrics.BaseMetric):
    def __init__(self, col=None, cdf=lambda x:x/(2*np.pi), args=(), period=2*np.pi, **kwargs):
        self.cdf = cdf
        self.args = args
        self.period = period
        assert self.cdf(0) == 0.0
        assert self.cdf(self.period) == 1.0
        super(KuiperMetric, self).__init__(col=col, **kwargs)
    def run(self, dataSlice, slicePoint=None):
        lat2 = dataSlice[self.latCol]
        lon2 = dataSlice[self.lonCol]
        data = np.sort(lat,lon)
        cdfv = self.cdf(data, *self.args)
        N = len(data)
        D = np.amax(cdfv-np.arange(N)/float(N)) + np.amax((np.arange(N)+1)/float(N)-cdfv)
        return D

Exception: Redefining metric __main__.KuiperMetric! (there are >1 metrics with the same name)

In [18]:
bundleList = []
sql = 'filter="r"'
metric = KuiperMetric('night')
slicer = slicers.HealpixSlicer(nside=64)
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql))


Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [20]:
outDir = 'temp'
conn = db.Database('simruns/minion_1012_sqlite.db')
resultsDb = db.ResultsDb(outDir=outDir)


In [21]:
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir)
mbg.runAll()
mbg.plotAll(closefigs=False)
conn.close()

Querying database None with constraint filter="r" for columns ['night', 'fieldRA', 'fieldDec']
Plotting figures with "filter="r"" constraint now.
Plotting complete.


/global/u2/h/husni/newcontrib/sims_maf/python/lsst/sims/maf/metricBundles/metricBundleGroup.py:269: UserWarning: One or more of the columns requested from the database was not available. Skipping constraint filter="r"
  ' Skipping constraint %s' % constraint)
/global/u2/h/husni/newcontrib/sims_maf/python/lsst/sims/maf/metricBundles/metricBundleGroup.py:273: UserWarning:  This means skipping metrics ['Kuiper night : r : HealpixSlicer']
  warnings.warn(' This means skipping metrics %s' % metricsSkipped)
/global/u2/h/husni/newcontrib/sims_maf/python/lsst/sims/maf/plots/plotHandler.py:534: UserWarning: MetricBundle (opsim_Kuiper_night_r_HEAL) has no attribute "metricValues". Either the values have not been calculated or they have been deleted.
  warnings.warn(msg)


In [23]:
conn.get_column_names()

{'Cloud': ['cloudID', 'c_date', 'cloud'],
 'Config': ['configID',
  'moduleName',
  'paramIndex',
  'paramName',
  'paramValue',
  'comment',
  'Session_sessionID',
  'nonPropID'],
 'Config_File': ['config_fileID', 'filename', 'data', 'Session_sessionID'],
 'Field': ['fieldID',
  'fieldFov',
  'fieldRA',
  'fieldDec',
  'fieldGL',
  'fieldGB',
  'fieldEL',
  'fieldEB'],
 'Log': ['logID', 'log_name', 'log_value', 'Session_sessionID'],
 'MissedHistory': ['missedHistID',
  'Session_sessionID',
  'filter',
  'expDate',
  'expMJD',
  'night',
  'lst',
  'Field_fieldID'],
 'ObsHistory': ['obsHistID',
  'Session_sessionID',
  'filter',
  'expDate',
  'expMJD',
  'night',
  'visitTime',
  'visitExpTime',
  'finRank',
  'finSeeing',
  'transparency',
  'airmass',
  'vSkyBright',
  'filtSkyBrightness',
  'rotSkyPos',
  'lst',
  'altitude',
  'azimuth',
  'dist2Moon',
  'solarElong',
  'moonRA',
  'moonDec',
  'moonAlt',
  'moonAZ',
  'moonPhase',
  'sunAlt',
  'sunAZ',
  'phaseAngle',
  'rScatte